## Drought Predictor

### Mission and Purpose

Most scientists and software applications which focus on predicting areas that are likely to be affected by droughts, through the measurements and analysis of indexes such as the Standardized Precipitation Index (SPI) or the Palmer Drought Index (PDI), are focused on the United States and Europe. Additionally, they tend to focus on one nation or a subset of that nation. If any global efforts are underway, they remain to be relatively unknown.

However, organizations have the data necessary to calculate the likelihood of droughts at locations around the world. For example, NASA has collected precipitation values at a global scale, which is the only information necessary to calculate the Standardized Precipitation Index. Thus, the purpose of my project is to create a Jupyter Notebook to determine where droughts are affecting the world in a readable format, such that this prototype will display the accurate information for the month of July in the year 2022 while allowing the scientific community to learn and build upon it.


### Methodology

I will be using an interactive map widget, provided by the ipywidgets, ipyleaflet, and leafmap Python packages, to show the status of locations in relation to the Standardized Precipitation Index. This index requires multiple precipitation values for each location in question. Thankfully, NASA readily provides this information from multiple years, with a new dataset available every month. As of now, the most recent dataset is from July 2022. My calculations used one dataset from every year for the years 2017 to 2022. However, the program was designed to be expandable, meaning that more and different datasets can be used, as long as they are from the same directory, due to the format of the code. You can find more information on this below.

### Drought Project: The Code

Now, I will code the project and create the interactive map. Numerous comments have been added to help others understand the design and functionality of the code.

The interactive map widget can be viewed all the way at the bottom. First, you must run all of the cells. If there are any unresolved inputs, open the command prompt (administrative) and download the files using `pip` (ex. `pip install ipyleaflet`). Note that the `math` and `csv` packages are already included.

In [1]:
## Imports ##
import ipyleaflet
from ipyleaflet import Map, AwesomeIcon, Marker, SearchControl, LayerGroup, CircleMarker, basemaps, basemap_to_tiles
import ipywidgets

import math
import csv


In [2]:
## SPI Calculator ##

# SPI is a widely accepted index to calculate the intensity of drought.
# Formula: SPI = (P - *P) / σp
# P: precipitation
# *P: mean precipitation
# σp: Standard deviation of precipitation

# This formula will take in a list of precipitation values.

def calculate_spi(precipitation):
    
    # Calculate Variables P and *P
    recent_precipitation = precipitation[-1]
    mean_precipitation = sum(precipitation) / len(precipitation)
    
    # Calculate Standard Deviation
    diff_list = []
    for val in precipitation:
        diff = val - mean_precipitation
        diff_list.append(diff * diff)
    sigma = sum(diff_list)
    under_sqr_root = sigma / len(precipitation)
    standard_deviation = math.sqrt(under_sqr_root)
    
    # Calculate SPI
    numerator = recent_precipitation - mean_precipitation
    denominator = standard_deviation
    if denominator == 0:
        spi = 0
    else:
        spi = numerator / denominator
    return spi

# Indexing the SPI Value
# 2.0 and higher: Extremely wet
# 1.5 ~ 1.99: Very Wet
# 1.0 ~ 1.49: Moderately Wet
# -.99 ~ .99: Near Normal
# -1.49 ~ -1.0: Moderately Dry
# -1.99 ~ -1.5: Severely Dry
# -2 and less: Extremely Dry

def spi_indexer(precipitation):
    spi = calculate_spi(precipitation)
    
    # Indexer
    if spi >= 2.0:
        index = "extremely wet"
    elif spi >=1.5:
        index = "very wet"
    elif spi >= 1:
        index = "moderately wet"
    elif spi > -1:
        index = "near normal"
    elif spi > -1.5:
        index = "moderately dry"
    elif spi > -2:
        index = "severely dry"
    elif spi <= -2:
        index = "extremely dry"
    else:
        index = "ERROR"
        print("Something went wrong when indexing. Please report this error.")
    
    return index

# Test Example
spi_indexer([0, 1.2, 0.4, 2])

'moderately wet'

In [3]:
## Read Data From CSV Files (Preprocess Data) ##

# Create a list containing the names of the CSV files. The last one should be the most recent.
file_names = ["precip_2017.csv", "precip_2018.csv", "precip_2019.csv", "precip_2020.csv", "precip_2021.csv", "precip_2022.csv"]

# Get the latitude and longitude values.
lat_vals = []
max_v = -88.75
while max_v <= 88.75:
    lat_vals.append(max_v)
    max_v += 2.5

# NOTE: Longitude values are degrees to the east and latitude values are degrees north.

lon_vals = []
max_v = 1.25
while max_v <= 358.75:
    lon_vals.append(max_v)
    max_v += 2.5

# Get the precipitation values from each year. The format will be as such:
# precip_dataset = [
# [file_1],
# [file_2],
# ...
# [most_recent_year]
# ]

# The files are formatted as such: Each row represents a latitude, starting at -88.75.
# Each column represents a longitude, starting at 1.25. Thus, if we pull a precipitation
# value from precip_dataset[1][8], it will be located at lat_vals[1] and long_vals[8].
precip_dataset = []

for file in file_names:
    
    annual_precip = []
    
    with open(f"{file}", mode ='r') as file:
    
        # reading the CSV file
        csvFile = csv.reader(file)
        
        # displaying the contents of the CSV file
        for lines in csvFile:
            annual_precip.append(lines)
    
    precip_dataset.append(annual_precip)

In [4]:
## Calculate and Plot ##

# Center for the map.
lat, lon = 27, -10

# Create the map outline.
theMap = ipyleaflet.Map(
    center=(lat,lon),
    zoom = 2,
    zoom_snap = 0.5,
    min_zoom = 2,
    basemap = ipyleaflet.basemaps.Esri.WorldStreetMap,
    draw_control=False,
    measure_control=False,
    fullscreen_control=False,
    attribution_control=True,
    toolbar_control = False)

# Create the icons for all possible SPI indexes.
icon_extremely_wet = AwesomeIcon(
    marker_color='darkblue',
    name = 'tint'
)

icon_very_wet = AwesomeIcon(
    marker_color='blue',
    name = 'tint'
)

icon_moderately_wet = AwesomeIcon(
    marker_color='lightblue',
    name = 'tint'
)

icon_near_normal = AwesomeIcon(
    marker_color='beige',
    name = 'tint'
)

icon_moderately_dry = AwesomeIcon(
    marker_color='orange',
    name = 'tint'
)

icon_severely_dry = AwesomeIcon(
    marker_color='lightred',
    name = 'tint'
)

icon_extremely_dry = AwesomeIcon(
    marker_color='darkred',
    name = 'tint'
)

# Calculate the SPI Index for each location and plot it on the map.
for lati in range(0, len(lat_vals), 2):
    for long in range(0, len(lon_vals), 2):
        
        precip_vals = []
        for year in range(len(file_names)):
            precip_vals.append(float(precip_dataset[year][lati][long]))

        spi_index = spi_indexer(precip_vals)
        
        if spi_index == "extremely wet":
            globals()[f"marker_{lati}{long}"] = Marker(location = (lat_vals[lati], [lon_vals[long] - 360 if lon_vals[long] > 180 else lon_vals[long]]), draggable = False, icon = icon_extremely_wet, title = "Extremely Wet", alt = "Extremely Wet")
            theMap.add_layer(globals()[f"marker_{lati}{long}"])
        
        elif spi_index == "very wet":
            globals()[f"marker_{lati}{long}"] = Marker(location = (lat_vals[lati], [lon_vals[long] - 360 if lon_vals[long] > 180 else lon_vals[long]]), draggable = False, icon = icon_very_wet, title = "Very Wet", alt = "Very Wet")
            theMap.add_layer(globals()[f"marker_{lati}{long}"])
            
        elif spi_index == "moderately wet":
            globals()[f"marker_{lati}{long}"] = Marker(location = (lat_vals[lati], [lon_vals[long] - 360 if lon_vals[long] > 180 else lon_vals[long]]), draggable = False, icon = icon_moderately_wet, title = "Moderately Wet", alt = "Moderately Wet")
            theMap.add_layer(globals()[f"marker_{lati}{long}"])
            
        elif spi_index == "near normal":
            globals()[f"marker_{lati}{long}"] = Marker(location = (lat_vals[lati], [lon_vals[long] - 360 if lon_vals[long] > 180 else lon_vals[long]]), draggable = False, icon = icon_near_normal, title = "Near Normal", alt = "Near Normal")
            theMap.add_layer(globals()[f"marker_{lati}{long}"])
            
        elif spi_index == "moderately dry":
            globals()[f"marker_{lati}{long}"] = Marker(location = (lat_vals[lati], [lon_vals[long] - 360 if lon_vals[long] > 180 else lon_vals[long]]), draggable = False, icon = icon_moderately_dry, title = "Moderately Dry", alt = "Moderately Dry")
            theMap.add_layer(globals()[f"marker_{lati}{long}"])
            
        elif spi_index == "severely dry":
            globals()[f"marker_{lati}{long}"] = Marker(location = (lat_vals[lati], [lon_vals[long] - 360 if lon_vals[long] > 180 else lon_vals[long]]), draggable = False, icon = icon_severely_dry, title = "Severely Dry", alt = "Severely Dry")
            theMap.add_layer(globals()[f"marker_{lati}{long}"])
            
        elif spi_index == "extremely dry":
            globals()[f"marker_{lati}{long}"] = Marker(location = (lat_vals[lati], [lon_vals[long] - 360 if lon_vals[long] > 180 else lon_vals[long]]), draggable = False, icon = icon_extremely_dry, title = "Extremely Dry", alt = "Extremely Dry")
            theMap.add_layer(globals()[f"marker_{lati}{long}"])

# Create the legend.
legend = ipyleaflet.LegendControl({"Extremely Wet":"#005998", "Very Wet":"#0098e2", "Moderately Wet":"#61eeff", "Near Normal": "#FFCC99", "Moderately Dry": "#FF9966", "Severely Dry": "#FAA", "Extremely Dry" : "#8B0000"}, name="Legend", position = "topright")
theMap.add_control(legend)

In [5]:
theMap 

Map(center=[27, -10], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_te…